## 2.1 Introduction

Assume there exist $n$ computing agents such as CPU or GPU, and each agent $i$ holds a local variable $x_i \in \mathbb{R}^d$. In distributed computing, one of the most important operation is **all-reduce** that let each agent $i$ achieve the global average of all local variables, i.e., $\bar{x} = \frac{1}{n}\sum_{i=1}^n x_i$. Such all-reduce operation is the step that requires significant communication among agents. It is usually the bottelneck that hinders the scalability of distributed systems.

There are many approaches to achieving all-reduce, some well-known methods are parameter-server or ring-allreduce. These approaches require global synchoronization across all agents and hence are regarded as **centralized** methods. They typically suffer from either significant bandwidth cost or high communication latency, see \[1\]. This motivates **decentralized** approaches in which the bottelneck of global synchonization can be relieved.

**Average consensus** is the most popular decentralized algorithm to achieve global average. Average consensus is based on partial averaging, in which each agent will compute the average of the nodes in its neighborhood. Repeating a series of such local actions can recover the global average of all the nodes in the network. Partial averaging was inspired by the similar processes in biology, animal behavior, and brain neuron [2, 3, 4]. No node maintains any global information in the entire updating process. The following figure illustrates decentralized communication. 

<center>
    <img src="./figures/decen_comm.png" alt="decen_comm" width="300"/>
    <br>
    <div style="color:orange; display: inline-block;
    color: #999;
    padding: 1px;">Fig.1 All agents exchange information over edges; they do not relay information. As an example, agents $i$ and $n$ collect information from their own neighborhoods. Other nodes do the same but not depicted.</div>
</center>

Average consensus is the foundation of decentralized optimization that we will discuss in later sections. In this section, we will describe the avearge consensus algorithm, discuss its convergence properties, and show how to implement it over the real distributed CPU clusters with BlueFog. We will also explore how the network topology will affect the convergence rate and communication efficiency of average consensus.

### 2.1.1 Organization

The organization of Section 2 is as follows: \[Will add link when these sections are ready\]

- **Sec. 2.2 Network topology**


- **Sec. 2.3 Combination matrix**


- **Sec. 2.4 Average consensus algorithm: implementation**


- **Sec. 2.5 Average consensus algorithm: convergence property**


- **Sec. 2.6 The influence of network topology**


- **Sec. 2.7 Advanced topic: push-sum average consensus**

### 2.1.2 Notation

- We let $(i,j)$ denote an edge of the network that starts from agent $i$ to agent $j$.

- Give a set $\mathcal{S}$, we let $|\mathcal{S}|$ denote its cardinality, i.e., the number of its elements.

### 2.1.3 Initialize BlueFog and test it

All contents in this section are displayed in Jupyter notebook, and all experimental examples are written with BlueFog and iParallel. Readers not familiar with how to run BlueFog in ipython notebook environment is encouraged to read Sec. [HelloWorld section] first. In the following codes, we will initialize BlueFog and test whether it works normally.

The output of `rc.ids ` should be a list from 0 to the number of processes minus one. The number of processes is the one you set in the `ibfrun start -np {X}`.

In [1]:
import ipyparallel as ipp

rc = ipp.Client(profile='bluefog')
rc.ids

Let each agent import necessary modules and then initialize BlueFog. You should be able to see the printed information like:  

> \[stdout:0\] Hello, I am 1 among 4 processes
> 
> ...

In [2]:
%%px
import numpy as np
import bluefog.torch as bf
import torch
from bluefog.common import topology_util
import networkx as nx

bf.init()
print(f"Hello, I am {bf.rank()} among {bf.size()} processes")

Push seed to each agent so that the simulation can be reproduced.

In [3]:
dview = rc[:] # A DirectView of all engines
dview.block=True

# Push the data into all workers
#   `dview.push({'seed': 2021}, block=True)`
# Or equivalently
dview['seed'] = 2021

After running the following code, you should be able to see the printed information like 

> \[stdout:0\] I received seed as value:  2021
> 
> ...

In [4]:
%%px
print("I received seed as value: ", seed)

[stdout:0] I received seed as value:  2021
[stdout:1] I received seed as value:  2021
[stdout:2] I received seed as value:  2021
[stdout:3] I received seed as value:  2021


Congratulations! Your BlueFog is initialized and tested successfully.